<a href="https://colab.research.google.com/github/zahrunyy7/sistem-cerdas_KNN/blob/main/Tugas_KNN_Sistem_Cerdas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
movies_df = pd.read_csv("/content/movies.csv",
                        usecols=['movieId', 'title'],
                        dtype={'movieId': 'int32', 'title': 'str'})

In [ ]:
rating_df = pd.read_csv(r'/content/ratings.csv',
                        usecols=['userId', 'movieId', 'rating'],
                        dtype={'userId': 'int32', 'movieId': 'int32', 'rating': 'float32'})

In [ ]:
movies_df.head(10)

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)
5,6,Heat (1995)
6,7,Sabrina (1995)
7,8,Tom and Huck (1995)
8,9,Sudden Death (1995)
9,10,GoldenEye (1995)


In [ ]:
movies_df.shape

(9742, 2)

In [ ]:
rating_df.head(10)

,userId,movieId,rating
0,1,1,4.000
1,1,3,4.000
2,1,6,4.000
3,1,47,5.000
4,1,50,5.000
5,1,70,3.000
6,1,101,5.000
7,1,110,4.000
8,1,151,5.000
9,1,157,5.000


In [ ]:
rating_df.shape

(100836, 3)

In [ ]:
df = pd.merge(rating_df, movies_df, on = 'movieId')
df

,userId,movieId,rating,title
0,1,1,4.000,Toy Story (1995)
1,1,3,4.000,Grumpier Old Men (1995)
2,1,6,4.000,Heat (1995)
3,1,47,5.000,Seven (a.k.a. Se7en) (1995)
4,1,50,5.000,"Usual Suspects, The (1995)"
...,...,...,...,...
100831,610,166534,4.000,Split (2017)
100832,610,168248,5.000,John Wick: Chapter Two (2017)
100833,610,168250,5.000,Get Out (2017)
100834,610,168252,5.000,Logan (2017)


In [ ]:
combine_movie_rating = df.dropna(axis=0, subset=['title'])

movie_ratingCount = (combine_movie_rating.
                     groupby(by=['title'])['rating'].
                     count().
                     reset_index().
                     rename(columns={'rating': 'totalRatingCount'})
                     [['title', 'totalRatingCount']]
                    )

movie_ratingCount

,title,totalRatingCount
0,'71 (2014),1
1,'Hellboy': The Seeds of Creation (2004),1
2,'Round Midnight (1986),2
3,'Salem's Lot (2004),1
4,'Til There Was You (1997),2
...,...,...
9714,eXistenZ (1999),22
9715,xXx (2002),24
9716,xXx: State of the Union (2005),5
9717,¡Three Amigos! (1986),26


In [ ]:
rating_with_totalRatingCount = combine_movie_rating.merge(movie_ratingCount, left_on='title', right_on='title', how='left')
rating_with_totalRatingCount.head(10)

,userId,movieId,rating,title,totalRatingCount
0,1,1,4.000,Toy Story (1995),215
1,1,3,4.000,Grumpier Old Men (1995),52
2,1,6,4.000,Heat (1995),102
3,1,47,5.000,Seven (a.k.a. Se7en) (1995),203
4,1,50,5.000,"Usual Suspects, The (1995)",204
5,1,70,3.000,From Dusk Till Dawn (1996),55
6,1,101,5.000,Bottle Rocket (1996),23
7,1,110,4.000,Braveheart (1995),237
8,1,151,5.000,Rob Roy (1995),44
9,1,157,5.000,Canadian Bacon (1995),11


In [ ]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
print(movie_ratingCount['totalRatingCount'].describe())

count   9719.000
mean      10.375
std       22.406
min        1.000
25%        1.000
50%        3.000
75%        9.000
max      329.000
Name: totalRatingCount, dtype: float64


In [ ]:
 popularity_threshold = 50
 rating_popular_movie = rating_with_totalRatingCount.query('totalRatingCount >= @popularity_threshold')
 rating_popular_movie.head()

,userId,movieId,rating,title,totalRatingCount
0,1,1,4.000,Toy Story (1995),215
1,1,3,4.000,Grumpier Old Men (1995),52
2,1,6,4.000,Heat (1995),102
3,1,47,5.000,Seven (a.k.a. Se7en) (1995),203
4,1,50,5.000,"Usual Suspects, The (1995)",204


In [ ]:
rating_popular_movie.shape

(41362, 5)

In [ ]:
movie_feature_df=rating_popular_movie.pivot_table(index='title', columns='userId', values='rating').fillna(0)
movie_feature_df.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You (1999),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,3.000,0.000,5.000,0.000,0.000,0.000,0.000,0.000
12 Angry Men (1957),0.000,0.000,0.000,5.000,0.000,0.000,0.000,0.000,0.000,0.000,...,5.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2001: A Space Odyssey (1968),0.000,0.000,0.000,0.000,0.000,0.000,4.000,0.000,0.000,0.000,...,0.000,0.000,5.000,0.000,0.000,5.000,0.000,3.000,0.000,4.500
28 Days Later (2002),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,3.500,0.000,5.000
300 (2007),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,3.000,...,0.000,0.000,0.000,0.000,3.000,0.000,0.000,5.000,0.000,4.000


In [ ]:
!pip install scipy
!pip install scikit-learn

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/commands/install.py", line 447, in run
    conflicts = self._determine_conflicts(to_install)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/commands/install.py", line 578, in _determine_conflicts
    return check_install_conflicts(to_install)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/operations/check.py", line 101, in check_install_conflicts
    package_set, _ = create_package_set_from_installed()
              

In [ ]:
from scipy.sparse import csr_matrix

movie_features_df_matrix = csr_matrix(movie_feature_df.values)

from sklearn.neighbors import NearestNeighbors

model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(movie_features_df_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [ ]:
movie_feature_df.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You (1999),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,3.000,0.000,5.000,0.000,0.000,0.000,0.000,0.000
12 Angry Men (1957),0.000,0.000,0.000,5.000,0.000,0.000,0.000,0.000,0.000,0.000,...,5.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2001: A Space Odyssey (1968),0.000,0.000,0.000,0.000,0.000,0.000,4.000,0.000,0.000,0.000,...,0.000,0.000,5.000,0.000,0.000,5.000,0.000,3.000,0.000,4.500
28 Days Later (2002),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,3.500,0.000,5.000
300 (2007),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,3.000,...,0.000,0.000,0.000,0.000,3.000,0.000,0.000,5.000,0.000,4.000


In [ ]:
query_index = np.random.choice(movie_feature_df.shape[0])
print(query_index)
distances, indices = model_knn.kneighbors(movie_feature_df.iloc[query_index,:].values.reshape(1, -1), n_neighbors=5)

2


In [ ]:
movie_feature_df.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You (1999),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,3.000,0.000,5.000,0.000,0.000,0.000,0.000,0.000
12 Angry Men (1957),0.000,0.000,0.000,5.000,0.000,0.000,0.000,0.000,0.000,0.000,...,5.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2001: A Space Odyssey (1968),0.000,0.000,0.000,0.000,0.000,0.000,4.000,0.000,0.000,0.000,...,0.000,0.000,5.000,0.000,0.000,5.000,0.000,3.000,0.000,4.500
28 Days Later (2002),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,3.500,0.000,5.000
300 (2007),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,3.000,...,0.000,0.000,0.000,0.000,3.000,0.000,0.000,5.000,0.000,4.000


In [ ]:
# Pastikan variabel 'rating_popular_movie' sudah ada dan berisi data
movie_features_df = rating_popular_movie.pivot_table(
    index='title',
    columns='userId',
    values='rating'
).fillna(0) # Mengganti nilai NaN (kosong) dengan 0

# Tampilkan 5 baris pertama untuk memastikan hasilnya ada
print(movie_features_df.head())

userId                              1     2     3     4     5     6     7    \
title                                                                         
10 Things I Hate About You (1999) 0.000 0.000 0.000 0.000 0.000 0.000 0.000   
12 Angry Men (1957)               0.000 0.000 0.000 5.000 0.000 0.000 0.000   
2001: A Space Odyssey (1968)      0.000 0.000 0.000 0.000 0.000 0.000 4.000   
28 Days Later (2002)              0.000 0.000 0.000 0.000 0.000 0.000 0.000   
300 (2007)                        0.000 0.000 0.000 0.000 0.000 0.000 0.000   

userId                              8     9     10   ...   601   602   603  \
title                                                ...                     
10 Things I Hate About You (1999) 0.000 0.000 0.000  ... 0.000 0.000 3.000   
12 Angry Men (1957)               0.000 0.000 0.000  ... 5.000 0.000 0.000   
2001: A Space Odyssey (1968)      0.000 0.000 0.000  ... 0.000 0.000 5.000   
28 Days Later (2002)              0.000 0.000 0.000  ...

In [ ]:
for i in range(0, len(distances.flatten())):
    if i == 0:
        # Mencetak judul film query (film yang dicari rekomendasinya)
        print('Recommendations for {0}:\n'.format(movie_features_df.index[query_index]))
    else:
        # Mencetak film rekomendasi (dimulai dari i=1 karena i=0 adalah film itu sendiri)
        print('{0}: {1}, with distance of {2}'.format(
              i,
              movie_features_df.index[indices.flatten()[i]],
              distances.flatten()[i]))

Recommendations for 2001: A Space Odyssey (1968):

1: Blade Runner (1982), with distance of 0.32926440238952637
2: Alien (1979), with distance of 0.43005305528640747
3: Apocalypse Now (1979), with distance of 0.4308894872665405
4: Aliens (1986), with distance of 0.4363347887992859
